In [16]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np


In [18]:
import os

folder = r"C:\Users\dsnat\Downloads\llm\Indian-monuments\images\test\lotus_temple"

print("Exists:", os.path.exists(folder))
print("Files:", os.listdir(folder))


Exists: True
Files: ['1.jfif', '10.jfif', '11.jfif', '12.jfif', '13.jfif', '14.jfif', '15.jfif', '16.jfif', '17.jfif', '18.jfif', '19.jfif', '2.jfif', '20.jfif', '21.jfif', '22.jfif', '23.jfif', '24.jfif', '25.jfif', '26.jfif', '27.jfif', '28.jfif', '29.jfif', '3.jfif', '30.jfif', '4.jfif', '5.jfif', '6.jfif', '7.jfif', '8.jfif', '9.jfif']


In [19]:
import os

folder = r"C:\Users\dsnat\Downloads\llm\Indian-monuments\images\test\lotus_temple"

for file in os.listdir(folder):
    old = os.path.join(folder, file)
    new = os.path.join(folder, file.lower())
    os.rename(old, new)

print("Converted to lowercase.")


Converted to lowercase.


In [20]:
import os

folder = r"C:\Users\dsnat\Downloads\llm\Indian-monuments\images\test\lotus_temple"

for file in os.listdir(folder):
    if file.endswith(".jfif"):
        old_path = os.path.join(folder, file)
        new_name = file.replace(".jfif", ".jpg")
        new_path = os.path.join(folder, new_name)
        os.rename(old_path, new_path)

print("Converted .jfif to .jpg")


Converted .jfif to .jpg


In [21]:
import os

base_test_dir = r"C:\Users\dsnat\Downloads\llm\Indian-monuments\images\test"

for class_name in os.listdir(base_test_dir):
    class_path = os.path.join(base_test_dir, class_name)
    
    if os.path.isdir(class_path):
        for file in os.listdir(class_path):
            if file.endswith(".jfif"):
                old_path = os.path.join(class_path, file)
                new_path = os.path.join(class_path, file.replace(".jfif", ".jpg"))
                os.rename(old_path, new_path)

print("All .jfif files converted to .jpg")


All .jfif files converted to .jpg


In [22]:
train_data = datasets.ImageFolder(train_dir)
test_data  = datasets.ImageFolder(test_dir)

print(train_data.classes == test_data.classes)


True


In [23]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [24]:
train_data = datasets.ImageFolder(train_dir, transform=transform)
test_data  = datasets.ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=32, shuffle=False)


In [25]:
from torchvision import models
import torch.nn as nn

model = models.resnet18(pretrained=True)

# Freeze feature extractor
for param in model.parameters():
    param.requires_grad = False

# Replace final layer
num_classes = len(train_data.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)


C:\Users\dsnat\anaconda3\envs\ml_lab\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dsnat\anaconda3\envs\ml_lab\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [27]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/5], Loss: 1.8482
Epoch [2/5], Loss: 0.7577
Epoch [3/5], Loss: 0.5026
Epoch [4/5], Loss: 0.3865
Epoch [5/5], Loss: 0.3253


In [29]:
torch.save(model.state_dict(), r"C:\Users\dsnat\Downloads\LLM project\resnet_monument_model.pth")